In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads, dropout=0.1):
        """
        멀티헤드 어텐션 모듈
        Args:
            embedding_dim: 임베딩 차원
            num_heads: 어텐션 헤드 개수
            dropout: 드롭아웃 비율
        """
        super(MultiHeadAttention, self).__init__()
        
        assert embedding_dim % num_heads == 0, "임베딩 차원은 헤드 개수로 나누어 떨어져야 합니다."
        
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.head_dim = embedding_dim // num_heads
        
        # 쿼리, 키, 밸류 가중치 행렬
        self.q_linear = nn.Linear(embedding_dim, embedding_dim)
        self.k_linear = nn.Linear(embedding_dim, embedding_dim)
        self.v_linear = nn.Linear(embedding_dim, embedding_dim)
        
        # 출력 가중치 행렬
        self.out_linear = nn.Linear(embedding_dim, embedding_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.scale = math.sqrt(self.head_dim)
        
    def forward(self, query, key, value, mask=None):
        """
        멀티헤드 어텐션 순전파
        Args:
            query: 쿼리 텐서 (batch_size, seq_length, embedding_dim)
            key: 키 텐서 (batch_size, seq_length, embedding_dim)
            value: 밸류 텐서 (batch_size, seq_length, embedding_dim)
            mask: 어텐션 마스크 (optional)
        Returns:
            output: 어텐션 결과 (batch_size, seq_length, embedding_dim)
            attention_weights: 어텐션 가중치
        """
        batch_size = query.size(0)
        
        # 선형 투영 및 헤드로 분할
        q = self.q_linear(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)  # (batch, num_heads, seq_len_q, head_dim)
        k = self.k_linear(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)  # (batch, num_heads, seq_len_k, head_dim)
        v = self.v_linear(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)  # (batch, num_heads, seq_len_v, head_dim)
        
        # 스케일드 닷-프로덕트 어텐션
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) / self.scale  # (batch, num_heads, seq_len_q, seq_len_k)
        
        # 마스크 적용 (필요한 경우)
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        
        # 어텐션 가중치 계산
        attention_weights = F.softmax(attention_scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # 가중치와 밸류의 곱
        attention_output = torch.matmul(attention_weights, v)  # (batch, num_heads, seq_len_q, head_dim)
        
        # 헤드 결합 및 출력 투영
        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, -1, self.embedding_dim)  # (batch, seq_len_q, embedding_dim)
        output = self.out_linear(attention_output)
        
        return output, attention_weights

class FeedForward(nn.Module):
    def __init__(self, embedding_dim, ff_dim, dropout=0.1):
        """
        피드포워드 네트워크 모듈
        Args:
            embedding_dim: 임베딩 차원
            ff_dim: 피드포워드 네트워크 내부 차원
            dropout: 드롭아웃 비율
        """
        super(FeedForward, self).__init__()
        
        self.linear1 = nn.Linear(embedding_dim, ff_dim)
        self.linear2 = nn.Linear(ff_dim, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """
        피드포워드 네트워크 순전파
        Args:
            x: 입력 텐서 (batch_size, seq_length, embedding_dim)
        Returns:
            출력 텐서 (batch_size, seq_length, embedding_dim)
        """
        # 첫 번째 선형 변환 및 활성화 함수 (ReLU)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        
        # 두 번째 선형 변환
        x = self.linear2(x)
        
        return x

class EncoderLayer(nn.Module):
    def __init__(self, embedding_dim, num_heads, ff_dim, dropout=0.1):
        """
        트랜스포머 인코더 레이어
        Args:
            embedding_dim: 임베딩 차원
            num_heads: 어텐션 헤드 개수
            ff_dim: 피드포워드 네트워크 내부 차원
            dropout: 드롭아웃 비율
        """
        super(EncoderLayer, self).__init__()
        
        self.self_attention = MultiHeadAttention(embedding_dim, num_heads, dropout)
        self.feed_forward = FeedForward(embedding_dim, ff_dim, dropout)
        
        self.layernorm1 = nn.LayerNorm(embedding_dim)
        self.layernorm2 = nn.LayerNorm(embedding_dim)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        인코더 레이어 순전파
        Args:
            x: 입력 텐서 (batch_size, seq_length, embedding_dim)
            mask: 어텐션 마스크 (optional)
        Returns:
            출력 텐서 (batch_size, seq_length, embedding_dim)
        """
        # 멀티헤드 셀프 어텐션 (서브레이어 1)
        attn_output, _ = self.self_attention(x, x, x, mask)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)  # 잔차 연결 및 레이어 정규화
        
        # 피드포워드 네트워크 (서브레이어 2)
        ff_output = self.feed_forward(out1)
        ff_output = self.dropout2(ff_output)
        out2 = self.layernorm2(out1 + ff_output)  # 잔차 연결 및 레이어 정규화
        
        return out2

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_layers=2, num_heads=8, ff_dim=2048, max_seq_length=100, dropout=0.1):
        """
        트랜스포머 인코더 모델
        Args:
            vocab_size: 어휘 크기 (성분 개수)
            embedding_dim: 임베딩 차원
            num_layers: 인코더 레이어 개수
            num_heads: 어텐션 헤드 개수
            ff_dim: 피드포워드 네트워크 내부 차원
            max_seq_length: 최대 시퀀스 길이
            dropout: 드롭아웃 비율
        """
        super(TransformerEncoder, self).__init__()
        
        # 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # 포지셔널 인코딩
        self.pos_encoding = self._create_positional_encoding(max_seq_length, embedding_dim)
        
        # 인코더 레이어 스택
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(embedding_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(embedding_dim)
        
    def _create_positional_encoding(self, seq_length, embedding_dim):
        """
        포지셔널 인코딩 생성
        Args:
            seq_length: 시퀀스 길이
            embedding_dim: 임베딩 차원
        Returns:
            포지셔널 인코딩 텐서
        """
        pos_encoding = np.zeros((seq_length, embedding_dim))
        positions = np.arange(seq_length)[:, np.newaxis]
        dimensions = np.arange(embedding_dim)[np.newaxis, :]
        
        angle_rates = 1 / np.power(10000, (2 * (dimensions // 2)) / embedding_dim)
        angle_rads = positions * angle_rates
        
        pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
        pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])
        
        return torch.FloatTensor(pos_encoding).unsqueeze(0)  # (1, seq_length, embedding_dim)
        
    def forward(self, x, mask=None):
        """
        트랜스포머 인코더 순전파
        Args:
            x: 입력 텐서 (batch_size, seq_length)
            mask: 어텐션 마스크 (optional)
        Returns:
            출력 텐서 (batch_size, seq_length, embedding_dim)
        """
        seq_length = x.size(1)
        
        # 임베딩 및 포지셔널 인코딩 적용
        x = self.embedding(x)  # (batch_size, seq_length, embedding_dim)
        x = x + self.pos_encoding[:, :seq_length, :].to(x.device)  # 포지셔널 인코딩 추가
        x = self.dropout(x)
        x = self.layernorm(x)  # 초기 정규화
        
        # 인코더 레이어 통과
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, mask)
        
        return x

In [2]:
class MultiLabelClassifier(nn.Module):
    def __init__(self, encoder, embedding_dim, num_labels=15):
        super(MultiLabelClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(embedding_dim, num_labels)
        
    def forward(self, x, mask=None):
        # 인코더의 출력 가져오기
        encoder_output = self.encoder(x, mask)
        
        # [CLS] 토큰 또는 평균 풀링 사용
        # 옵션 1: 첫 번째 토큰 ([CLS]) 사용
        # pooled_output = encoder_output[:, 0]
        
        # 옵션 2: 평균 풀링
        pooled_output = torch.mean(encoder_output, dim=1)
        
        # 분류기에 통과
        logits = self.classifier(pooled_output)
        
        return logits